## comparing 5/15 and 6/27

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
from pathlib import Path
import scipy

# load files
serverPath = Path(r'Z:\Subjects\AL_0023\2023-05-15\1')

timeFile = serverPath / 'cameraFrameTimes.npy'
frameTimes = np.squeeze(np.load(timeFile))[::2] # every other frame - we want blue only
svdTemp = np.load(serverPath / 'corr/50_svdTemporalComponents_corr.npy')
svdSpat = np.load(serverPath / 'blue/50_svdSpatialComponents.npy')
meanImage = np.load(serverPath / 'blue/meanImage.npy')
stimOnTimes = np.squeeze(np.load(r'Z:\Subjects\AL_0023\2023-05-15\3\linTestOnTimes.npy'))
stimContrasts = np.squeeze(np.load(r'Z:\Subjects\AL_0023\2023-05-15\3\linTestContrasts.npy'))
stimDurations = np.squeeze(np.load(r'Z:\Subjects\AL_0023\2023-05-15\3\linTestDurations.npy'))

# files from day 2 

serverPath2 = Path(r'Z:\Subjects\AL_0023/2023-06-27/1')

timeFile2 = serverPath2 / 'cameraFrameTimes.npy'
frameTimes2 = np.squeeze(np.load(timeFile2))[::2] # every other frame - we want blue only
svdTemp2 = np.load(serverPath2 / 'corr/50_svdTemporalComponents_corr.npy')
svdSpat2 = np.load(serverPath2 / 'blue/50_svdSpatialComponents.npy')
meanImage2 = np.load(serverPath2 / 'blue/meanImage.npy')
stimOnTimes2 = np.squeeze(np.load(r'Z:\Subjects\AL_0023/2023-06-27\3\linTestOnTimes.npy'))
stimContrasts2 = np.squeeze(np.load(r'Z:\Subjects\AL_0023/2023-06-27\3\linTestContrasts.npy'))
stimDurations2 = np.squeeze(np.load(r'Z:\Subjects\AL_0023\2023-06-27\3\linTestDurations.npy'))


In [ ]:
# creating arrays tht correspond to the max and min stim contrasts over time
uniqueContrasts = np.unique(stimContrasts)
maxUnique = max(uniqueContrasts)
minUnique = min(uniqueContrasts)
maxContrasts = np.where(np.array(stimContrasts) == (maxUnique))[0]
minContrasts = np.where(np.array(stimContrasts) == (minUnique))[0]
stimOnMax = stimOnTimes[maxContrasts]
stimOnMin = stimOnTimes[minContrasts]

uniqueContrasts2 = np.unique(stimContrasts2)
maxUnique2 = max(uniqueContrasts2)
maxContrasts2 = np.where(np.array(stimContrasts2) == maxUnique2)[0]
stimOnMax2 = stimOnTimes2[maxContrasts2]

# useful tools:
# np.unique
# np.argwhere
# (for finding duration AND contrast: np.intersect1d)

# interp function: get svdTemp at any time
getTempComp = scipy.interpolate.interp1d(frameTimes, svdTemp, axis=0)
getTempComp2 = scipy.interpolate.interp1d(frameTimes2, svdTemp2, axis=0)

trial_time_max_2 = [np.linspace(i-0.1, i+0.6, 100) for i in stimOnMax2]
trial_time_max_2 = np.array(trial_time_max_2)
trial_time_max = [np.linspace(i-0.1, i+0.6, 100) for i in stimOnMax]
trial_time_max = np.array(trial_time_max)
trial_activity_max_2 = getTempComp2(trial_time_max_2)

In [ ]:
trial_activity_max_reshaped = np.mean(trial_activity_max, axis=0)
spatial = svdSpat.reshape(560*560, -1)
videoMax = spatial @ trial_activity_max_reshaped.T
videoMax = videoMax.reshape(560, 560, -1)

In [ ]:
#testing that I made the video of the max contrast/time correctly
n_cols = 5
n_rows = 10
f = plt.figure(figsize=(n_cols*2, n_rows*2))
gs = mpl.gridspec.GridSpec(n_rows, n_cols)
for i in range(50):
    ax = plt.subplot(gs[i])
    plt.imshow(videoMax[:, :, i*2], clim=[-3000, 3000])
    plt.xticks([])
    plt.yticks([])
    #plt.title(f't = {trial_time_all[i*2]}s')
    # plt.colorbar()
    
#testing that I made the video of the max contrast/time correctly, 6/27 data
n_cols = 5
n_rows = 10
f = plt.figure(figsize=(n_cols*2, n_rows*2))
gs = mpl.gridspec.GridSpec(n_rows, n_cols)
for i in range(50):
    ax = plt.subplot(gs[i])
    plt.imshow(videoMax2[:, :, i*2], clim=[-3000, 3000])
    plt.xticks([])
    plt.yticks([])
    #plt.title(f't = {trial_time_all[i*2]}s')
    # plt.colorbar()

In [ ]:
# lets plot a pixel over time 
n_cols = 1
n_rows = 1
f = plt.figure(figsize=(n_cols*2, n_rows*2))
gs = mpl.gridspec.GridSpec(n_rows, n_cols)
ax = plt.subplot(gs[0])
plt.imshow(videoMax[:, :, (locationMax)], clim=[-3000, 3000])
plt.scatter([350],[400], c="orange", marker="v")
plt.xticks([])
plt.yticks([])
ax.set_title("Max")

#plot pixel over time, 6/27 data
n_cols = 1
n_rows = 1
f = plt.figure(figsize=(n_cols*2, n_rows*2))
gs = mpl.gridspec.GridSpec(n_rows, n_cols)
ax = plt.subplot(gs[0])
plt.imshow(videoMax[:, :, (locationMax)], clim=[-3000, 3000])
plt.scatter([399],[430], c="orange", marker="v")
plt.xticks([])
plt.yticks([])
ax.set_title("Max")

In [ ]:
# correction with all contrasts, max duration

locContrasts = np.unique(stimContrasts)
locDurations = np.unique(stimDurations)
maxDuration = max(locDurations)
theseIndexesDur = np.squeeze(np.argwhere(stimDurations == maxDuration))
    
videos = []
spatial = svdSpat.reshape(560*560, -1)
timeScale = np.linspace(-0.1, 1.2, 100)

for contrast in locContrasts:
    theseIndexesCon = np.squeeze(np.argwhere(stimContrasts == contrast))
    
    indexes = np.intersect1d(theseIndexesCon, theseIndexesDur)
    stim = stimOnTimes[indexes]
    
    trial_time = [np.linspace(i-0.1, i+1.2, 100) for i in stim]
    trial_activity = getTempComp(trial_time)

    reshape = np.mean(trial_activity, axis=0)
    video = spatial @ reshape.T
    video = video.reshape(560, 560, -1)
    videos.append(video)
    
videos = np.array(videos)

colors = ['red', 'blue', 'orange', 'pink', 'purple']
for j in range(0,5):
    frame_raw = videos[j,330:350,380:400, :]
    # print(frame_raw.shape)
    frame = np.mean(frame_raw, axis=(0, 1))
    # print(frame.shape)
    minActivity = min(frame)
    if minActivity < 0:
        #frame = [(x+minActivity*-1) for x in frame]
        frame += minActivity*-1
    f0 = np.mean(frame[:6])
    frame = [(x-f0)/(f0+1) for x in frame]
    plt.plot(timeScale, frame, marker='.', c=colors[j])
plt.legend(["0.062", ".125", '.25', '.5', '1'])
plt.xlabel("Time (s)")
plt.ylabel("Activity")
plt.title("Activity for Contrasts, Max Duration")

# correction with 6/27 data 

locContrasts2 = np.unique(stimContrasts2)
locDurations2 = np.unique(stimDurations2)
maxDuration2 = max(locDurations2)
theseIndexesDur2 = np.squeeze(np.argwhere(stimDurations2 == maxDuration2))
    
videos2 = []
spatial2 = svdSpat2.reshape(560*560, -1)
timeScale2 = np.linspace(-0.1, 1.2, 100)

for contrast2 in locContrasts2:
    theseIndexesCon2 = np.squeeze(np.argwhere(stimContrasts2 == contrast2))
    
    indexes2 = np.intersect1d(theseIndexesCon2, theseIndexesDur2)
    stim2 = stimOnTimes2[indexes2]
    
    trial_time2 = [np.linspace(i-0.1, i+1.2, 100) for i in stim2]
    trial_activity2 = getTempComp2(trial_time2)

    reshape2 = np.mean(trial_activity2, axis=0)
    video2 = spatial2 @ reshape2.T
    video2 = video2.reshape(560, 560, -1)
    videos2.append(video2)
    
videos2 = np.array(videos2)

colors2 = ['red', 'blue', 'orange', 'pink', 'purple']
for j in range(0,5):
    frame_raw2 = videos2[j,389:409,420:440, :]
    # print(frame_raw.shape)
    frame2 = np.mean(frame_raw2, axis=(0, 1))
    # print(frame.shape)
    minActivity2 = min(frame2)
    if minActivity2 < 0:
        #frame = [(x+minActivity*-1) for x in frame]
        frame2 += minActivity2*-1
    f0 = np.mean(frame2[:6])
    frame2 = [(x-f0)/(f0+1) for x in frame2]
    plt.plot(timeScale2, frame2, marker='.', c=colors2[j])
plt.legend(["0.062", ".125", '.25', '.5', '1'])
plt.xlabel("Time (s)")
plt.ylabel("Activity")
plt.title("Activity for Contrasts, Max Duration 6/27")

In [ ]:
#plotting contrasts against each other 5/15 vs 6/27
n_cols = 2
n_rows = 5
f = plt.figure(figsize=(n_cols*4, n_rows*4))
gs = mpl.gridspec.GridSpec(n_rows, n_cols)
for i in range(n_rows):
    ax = plt.subplot(gs[i,0])
    frame_raw = videos[i,330:350,380:400, :]
    frame = np.mean(frame_raw, axis=(0, 1))
    minActivity = min(frame)
    if minActivity < 0:
        frame += minActivity*-1
    f0 = np.mean(frame[:6])
    frame = [(x-f0)/(f0+1) for x in frame]
    plt.plot(timeScale, frame, marker='.', c=colors[i])
    plt.title("Contrast value: " + str(locContrasts[i]))
    plt.xlabel("time (s)")
    plt.ylabel("relative activity")
    
    ax = plt.subplot(gs[i,1])
    frame_raw2 = videos2[i,389:409,420:440, :]
    frame2 = np.mean(frame_raw2, axis=(0, 1))
    minActivity2 = min(frame2)
    if minActivity2 < 0:
        frame2 += minActivity2*-1
    f02 = np.mean(frame2[:6])
    frame2 = [(x-f02)/(f02+1) for x in frame2]
    plt.plot(timeScale2, frame2, marker='.', c=colors2[i])
    plt.title("Contrast value: " + str(locContrasts[i]))
    plt.xlabel("time (s)")
    plt.ylabel("relative activity")

f.tight_layout()

In [ ]:
# bar graphs. contrast v mean activity, under different durations 

# loop through durations
# for each durations, create arrays of contrast + mean activity 
# bar graph 

locDurations = np.unique(stimDurations)
locContrasts = np.unique(stimContrasts)
collectAverages = []
collectAveragesContrast = []
for duration in locDurations:
    theseIndexesDur = np.squeeze(np.argwhere(stimDurations == duration))
    for contrast in locContrasts: #for each contrast
        contrastIndeces = [j for j,value in enumerate(stimContrasts) if (value==contrast)] #find indeces where specfic contrast was used
        indexes = np.intersect1d(contrastIndeces, theseIndexesDur) #find indeces where its both the duration we want and the contrast we want
        stim = stimOnTimes[indexes] #make an array of only the indeces found above
        trial_time = [np.linspace(k-0.1, k+.6, 100) for k in stim]
        trial_activity = getTempComp(trial_time)
        
        #correct activity so its not negative 
        minActivity = np.min(trial_activity)
        if minActivity < 0:
            trial_activity += minActivity*-1
            
        averageForContrast = np.mean(trial_activity)
        collectAveragesContrast.append(averageForContrast)
    collectAverages.append(collectAveragesContrast)
    collectAveragesContrast = []
collectAverages = np.array(collectAverages)

n_cols = 2
n_rows = 3
f = plt.figure(figsize=(n_cols*4, n_rows*4))
gs = mpl.gridspec.GridSpec(n_rows, n_cols)
for i in range(3):
    ax = plt.subplot(gs[i,0])
    plt.bar(range(0,5), collectAverages[i,:], width = .8)
    
ax = plt.subplot(111)
im = ax.imshow(collectAverages, cmap='viridis')
color = plt.colorbar(im)
color.ax.set_ylabel("avg activity")
ax.set_xticks(np.arange(len(locContrasts)), labels=locContrasts)
ax.set_xlabel("contrast value")
ax.set_yticks(np.arange(len(locDurations)), labels=np.flip(locDurations))
ax.set_ylabel("duration value (s)")
ax.set_title("activity levels of contrast and duration")

#averages 6/27

# bar graphs. contrast v mean activity, under different durations 

# loop through durations
# for each durations, create arrays of contrast + mean activity 
# bar graph 

locDurations2 = np.unique(stimDurations2)
locContrasts2 = np.unique(stimContrasts2)
collectAverages2 = []
collectAveragesContrast2 = []
for duration2 in locDurations2:
    theseIndexesDur2 = np.squeeze(np.argwhere(stimDurations2 == duration2))
    for contrast2 in locContrasts2: #for each contrast
        contrastIndeces2 = [j for j,value in enumerate(stimContrasts2) if (value==contrast2)] #find indeces where specfic contrast was used
        indexes2 = np.intersect1d(contrastIndeces2, theseIndexesDur2) #find indeces where its both the duration we want and the contrast we want
        stim2 = stimOnTimes2[indexes2] #make an array of only the indeces found above
        trial_time2 = [np.linspace(k-0.1, k+.6, 100) for k in stim2]
        trial_activity2 = getTempComp2(trial_time2)
        
        #correct activity so its not negative 
        minActivity2 = np.min(trial_activity2)
        if minActivity2 < 0:
            trial_activity2 += minActivity2*-1
            
        averageForContrast2 = np.mean(trial_activity2)
        collectAveragesContrast2.append(averageForContrast2)
    collectAverages2.append(collectAveragesContrast2)
    collectAveragesContrast2 = []
collectAverages2 = np.array(collectAverages2)

ax = plt.subplot(111)
im = ax.imshow(collectAverages2, cmap='jet')
color = plt.colorbar(im)
color.ax.set_ylabel("avg activity")
ax.set_xticks(np.arange(len(locContrasts2)), labels=locContrasts2)
ax.set_xlabel("contrast value")
ax.set_yticks(np.arange(len(locDurations2)), labels=np.flip(locDurations2))
ax.set_ylabel("duration value (s)")
ax.set_title("activity levels of contrast and duration")